### This notebook opens and select some features from two datasets: Carseats and CreditCardBehavior

### The first is meant to be used for regression and classification problems and the second for clustering problems.

Source: 
- Carseats: https://github.com/selva86/datasets/blob/master/Carseats.csv and https://rdrr.io/cran/ISLR/man/Carseats.html
- Credit Card Behavior: https://www.kaggle.com/datasets/arjunbhasin2013/ccdata?resource=download

# Carseats

Description of features:

- **Sales - OUR TARGET**
    Unit sales (in thousands) at each location
    
- **CompPrice**
    Price charged by competitor at each location
    
- **Income**
    Community income level (in thousands of dollars)
    
- **Advertising**
    Local advertising budget for company at each location (in thousands of dollars)
    
- **Population**
    Population size in region (in thousands)
    
- **Price**
    Price company charges for car seats at each site
    
- **ShelveLoc**
    A factor with levels Bad, Good and Medium indicating the quality of the shelving location for the car seats at each site
    
- **Age**
    Average age of the local population
    
- **Education**
    Education level at each location
    
- **Urban**
    A factor with levels No and Yes to indicate whether the store is in an urban or rural location
    
- **US**
    A factor with levels No and Yes to indicate whether the store is in the US or not


In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np

In [2]:
carseats = pd.read_csv('Carseats.csv')
carseats.head()

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No


In [3]:
size_before = carseats.shape[0]

# Simple cleaning
carseats.dropna(inplace=True)
carseats.drop_duplicates(inplace=True)

size_after = carseats.shape[0]
print(f"Dropped {size_before-size_after} empty or duplicated rows")

Dropped 0 empty or duplicated rows


In [4]:
# Look for correlation to choose some features
carseats.corr()

,Sales,CompPrice,Income,Advertising,Population,Price,Age,Education
Sales,1.000000,0.064079,0.151951,0.269507,0.050471,-0.444951,-0.231815,-0.051955
CompPrice,0.064079,1.000000,-0.080653,-0.024199,-0.094707,0.584848,-0.100239,0.025197
Income,0.151951,-0.080653,1.000000,0.058995,-0.007877,-0.056698,-0.004670,-0.056855
Advertising,0.269507,-0.024199,0.058995,1.000000,0.265652,0.044537,-0.004557,-0.033594
Population,0.050471,-0.094707,-0.007877,0.265652,1.000000,-0.012144,-0.042663,-0.106378
Price,-0.444951,0.584848,-0.056698,0.044537,-0.012144,1.000000,-0.102177,0.011747
Age,-0.231815,-0.100239,-0.004670,-0.004557,-0.042663,-0.102177,1.000000,0.006488
Education,-0.051955,0.025197,-0.056855,-0.033594,-0.106378,0.011747,0.006488,1.000000


In [5]:
carseats.describe()

,Sales,CompPrice,Income,Advertising,Population,Price,Age,Education
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000
mean,7.496325,124.975000,68.657500,6.635000,264.840000,115.795000,53.322500,13.900000
std,2.824115,15.334512,27.986037,6.650364,147.376436,23.676664,16.200297,2.620528
min,0.000000,77.000000,21.000000,0.000000,10.000000,24.000000,25.000000,10.000000
25%,5.390000,115.000000,42.750000,0.000000,139.000000,100.000000,39.750000,12.000000
50%,7.490000,125.000000,69.000000,5.000000,272.000000,117.000000,54.500000,14.000000
75%,9.320000,135.000000,91.000000,12.000000,398.500000,131.000000,66.000000,16.000000
max,16.270000,175.000000,120.000000,29.000000,509.000000,191.000000,80.000000,18.000000


In [6]:
# Create function to classify values into ranges Low, Medium and High

def value_to_class(medium, high, value):
    if value<=medium:
        return "Low"
    if value>medium and value<high:
        return "Medium"
    return "High"

In [7]:
# Here I choose medium as first quartile (the first 25%) and high as third quartile (above the 75%)

carseats_categ = carseats.copy()
carseats_categ['SalesClass'] = carseats.Sales.apply(lambda x: value_to_class(5.4,9.3,x))
carseats_categ['AdvertisingClass'] = carseats.Advertising.apply(lambda x: value_to_class(0,12,x))
carseats_categ['PriceClass'] = carseats.Price.apply(lambda x: value_to_class(100,131,x))

In [8]:
# Saving as excel - SalesClass is our target

categ_feats = ['ShelveLoc','Urban', 'US', 'AdvertisingClass', 'PriceClass', 'SalesClass']
carseats_categ[categ_feats].to_excel('CarseatsCategoricalFeatures.xlsx')


In [9]:
# Now a numerical version of the same dataframe

# I'll trasnform categorical features to numerical
print(carseats.ShelveLoc.unique())
print(carseats.Urban.unique())
print(carseats.US.unique())

['Bad' 'Good' 'Medium']
['Yes' 'No']
['Yes' 'No']


In [10]:
carseats_num = carseats.copy()

# Converting type of columns to category
carseats_num['ShelveLoc'] = carseats['ShelveLoc'].astype('category')
carseats_num['Urban'] = carseats['Urban'].astype('category')
carseats_num['US'] = carseats['US'].astype('category')

# Assigning numerical values and storing it in another columns
carseats_num['ShelveLocNum'] = carseats_num['ShelveLoc'].cat.codes
carseats_num['UrbanNum'] = carseats_num['Urban'].cat.codes
carseats_num['USNum'] = carseats_num['US'].cat.codes
carseats_num

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US,ShelveLocNum,UrbanNum,USNum
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes,0,1,1
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes,1,1,1
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes,2,1,1
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes,2,1,1
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,12.57,138,108,17,203,128,Good,33,14,Yes,Yes,1,1,1
396,6.14,139,23,3,37,120,Medium,55,11,No,Yes,2,0,1
397,7.41,162,26,12,368,159,Medium,40,18,Yes,Yes,2,1,1
398,5.94,100,79,7,284,95,Bad,50,12,Yes,Yes,0,1,1


In [11]:
# Adding some features here (Income and Age) - Sales is our target

num_feats = ['ShelveLocNum','UrbanNum', 'USNum', 'Advertising', 'Price', 'Income', 'Age', 'Sales']
carseats_num[num_feats].to_excel('CarseatsCNumericalFeatures.xlsx')

# Credit Card Behavior

Description of features, all of them are numerical:

- CUST_ID : Identification of Credit Card holder (Categorical)
- BALANCE : Balance amount left in their account to make purchases (
- BALANCE_FREQUENCY : How frequently the Balance is updated, score between 0 and 1 (1 = frequently updated, 0 = not frequently updated)
- PURCHASES : Amount of purchases made from account
- ONEOFF_PURCHASES : Maximum purchase amount done in one-go
- INSTALLMENTS_PURCHASES : Amount of purchase done in installment
- CASH_ADVANCE : Cash in advance given by the user
- PURCHASES_FREQUENCY : How frequently the Purchases are being made, score between 0 and 1 (1 = frequently purchased, 0 = not frequently purchased)
- ONEOFFPURCHASESFREQUENCY : How frequently Purchases are happening in one-go (1 = frequently purchased, 0 = not frequently purchased)
- PURCHASESINSTALLMENTSFREQUENCY : How frequently purchases in installments are being done (1 = frequently done, 0 = not frequently done)
- CASHADVANCEFREQUENCY : How frequently the cash in advance being paid
- CASHADVANCETRX : Number of Transactions made with "Cash in Advanced"
- PURCHASES_TRX : Numbe of purchase transactions made
- CREDIT_LIMIT : Limit of Credit Card for user
- PAYMENTS : Amount of Payment done by user
- MINIMUM_PAYMENTS : Minimum amount of payments made by user
- PRCFULLPAYMENT : Percent of full payment paid by user
- TENURE : Tenure of credit card service for user

Source: https://www.kaggle.com/datasets/arjunbhasin2013/ccdata?resource=download

In [12]:
data = pd.read_csv('CreditCardBehavior.csv')
data.head()

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,C10001,40.900749,0.818182,95.40,0.00,95.4,0.000000,0.166667,0.000000,0.083333,0.000000,0,2,1000.0,201.802084,139.509787,0.000000,12
1,C10002,3202.467416,0.909091,0.00,0.00,0.0,6442.945483,0.000000,0.000000,0.000000,0.250000,4,0,7000.0,4103.032597,1072.340217,0.222222,12
2,C10003,2495.148862,1.000000,773.17,773.17,0.0,0.000000,1.000000,1.000000,0.000000,0.000000,0,12,7500.0,622.066742,627.284787,0.000000,12
3,C10004,1666.670542,0.636364,1499.00,1499.00,0.0,205.788017,0.083333,0.083333,0.000000,0.083333,1,1,7500.0,0.000000,NaN,0.000000,12
4,C10005,817.714335,1.000000,16.00,16.00,0.0,0.000000,0.083333,0.083333,0.000000,0.000000,0,1,1200.0,678.334763,244.791237,0.000000,12


In [13]:
size_before = data.shape[0]

# Simple cleaning
data.dropna(inplace=True)
data.drop_duplicates(inplace=True)

size_after = data.shape[0]
print(f"Dropped {size_before-size_after} empty or duplicated rows")

Dropped 314 empty or duplicated rows


In [14]:
# Look for correlation to choose some features
data.corr()

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
BALANCE,1.000000,0.310140,0.176083,0.159985,0.122109,0.495586,-0.088459,0.063832,-0.069582,0.445307,0.382388,0.147887,0.535518,0.322830,0.398669,-0.333594,0.066987
BALANCE_FREQUENCY,0.310140,1.000000,0.122635,0.095254,0.114739,0.089036,0.228158,0.187467,0.184159,0.181132,0.133265,0.183095,0.087682,0.039169,0.132519,-0.156961,0.104714
PURCHASES,0.176083,0.122635,1.000000,0.916780,0.679259,-0.053760,0.393000,0.497384,0.316025,-0.124863,-0.070277,0.688732,0.358425,0.606782,0.093842,0.176447,0.084545
ONEOFF_PURCHASES,0.159985,0.095254,0.916780,1.000000,0.329650,-0.033244,0.265460,0.524514,0.128380,-0.086413,-0.048705,0.545313,0.320613,0.570850,0.048741,0.129890,0.063400
INSTALLMENTS_PURCHASES,0.122109,0.114739,0.679259,0.329650,1.000000,-0.066099,0.441193,0.211829,0.511130,-0.136455,-0.076716,0.626083,0.258057,0.385391,0.132156,0.178440,0.083346
CASH_ADVANCE,0.495586,0.089036,-0.053760,-0.033244,-0.066099,1.000000,-0.218566,-0.090111,-0.178873,0.629030,0.656911,-0.078449,0.304161,0.459342,0.140094,-0.158566,-0.073090
PURCHASES_FREQUENCY,-0.088459,0.228158,0.393000,0.265460,0.441193,-0.218566,1.000000,0.502123,0.862338,-0.316771,-0.208749,0.567168,0.121372,0.100715,0.002979,0.305393,0.052161
ONEOFF_PURCHASES_FREQUENCY,0.063832,0.187467,0.497384,0.524514,0.211829,-0.090111,0.502123,1.000000,0.143911,-0.118225,-0.073384,0.544364,0.295432,0.244346,-0.030357,0.154073,0.080437
PURCHASES_INSTALLMENTS_FREQUENCY,-0.069582,0.184159,0.316025,0.128380,0.511130,-0.178873,0.862338,0.143911,1.000000,-0.268924,-0.172811,0.529192,0.063421,0.083424,0.030036,0.249547,0.065779
CASH_ADVANCE_FREQUENCY,0.445307,0.181132,-0.124863,-0.086413,-0.136455,0.629030,-0.316771,-0.118225,-0.268924,1.000000,0.799593,-0.136913,0.132841,0.182340,0.098844,-0.260001,-0.140038


### Here we can't choose the most correlated to the target (since we didn't define one), but let's be logical about which features select

In [15]:
# We don't need the ID from the client and let's take just some simple features

credit_df = data[['BALANCE', 'BALANCE_FREQUENCY', 'PURCHASES',
       'PURCHASES_FREQUENCY', 'PURCHASES_TRX', 'CREDIT_LIMIT', 'PAYMENTS',
        'PRC_FULL_PAYMENT']]

In [16]:
# For educational purposes, the dataset don't need to be so long
credit_df = credit_df.loc[:399,:]

In [17]:
credit_df.to_excel('CreditCardClean.xlsx')